In [1]:
import tensorflow as tf 
import numpy as np
import time
def get_f1(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
pretrainedmodel_files = ["/kaggle/input/modelaugmentat/model_segmentare_aug.h5", "/kaggle/input/clasificare-neaugmentat/model_neaugmentat_clasificare.h5"]
A = tf.keras.models.load_model("/kaggle/input/modelaugmentat/model_segmentare_aug.h5" , custom_objects={"get_f1": get_f1 })
B = tf.keras.models.load_model("/kaggle/input/clasificareaugmentat/model_augmentat_clasificare.h5")


2022-12-21 14:38:04.514863: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 14:38:04.652392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 14:38:04.653401: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 14:38:04.655203: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [2]:
import tensorflow
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras import Sequential
import keras
import keras.backend as K
from keras.layers import Concatenate, Reshape, Permute

input_a = keras.Input(shape=(256,256,1))
input_b = keras.Input(shape=(400,400,3))

masked_img = A(input_a)
masked_img = Resizing(400,400)(masked_img)
masked_img = Reshape((-1,400,400))(masked_img)
masked_img = Resizing(3,400)(masked_img)
masked_img = Permute((2,3,1))(masked_img)
second_img = B(masked_img)

model_merged = tf.keras.Model(input_a, second_img, name="finalModel")

model_merged.summary()

Model: "finalModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 1)]     0         
_________________________________________________________________
model (Functional)           (None, 256, 256, 1)       31401345  
_________________________________________________________________
resizing (Resizing)          (None, 400, 400, 1)       0         
_________________________________________________________________
reshape (Reshape)            (None, 1, 400, 400)       0         
_________________________________________________________________
resizing_1 (Resizing)        (None, 3, 400, 400)       0         
_________________________________________________________________
permute (Permute)            (None, 400, 400, 3)       0         
_________________________________________________________________
sequential (Sequential)      (None, 3)                 7

In [3]:
#train the model
import cv2
import tensorflow as tf
from glob import glob
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from random import randint
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image
import os, shutil

os.mkdir('/kaggle/working/images')
os.mkdir('/kaggle/working/images/benign')
os.mkdir('/kaggle/working/images/malignant')
os.mkdir('/kaggle/working/images/normal')
PATH = "../input/augmented-data/BUSI - Augmented Dataset"
directory = os.path.join(PATH,"benign/")
for filename in os.listdir(directory):
    if "mask" not in filename: 
         shutil.copy2(os.path.join(PATH,f"benign/{filename}"), f"/kaggle/working/images/benign/{filename}")

directory = os.path.join(PATH,"malignant/")
for filename in os.listdir(directory):
    if "mask" not in filename: 
         shutil.copy2(os.path.join(PATH,f"malignant/{filename}"), f"/kaggle/working/images/malignant/{filename}")

directory = os.path.join(PATH,"normal/")
for filename in os.listdir(directory):
    if "mask" not in filename: 
         shutil.copy2(os.path.join(PATH,f"normal/{filename}"), f"/kaggle/working/images/normal/{filename}")


In [ ]:
import cv2
import tensorflow as tf
from glob import glob
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from random import randint
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image
import os, shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
batch_size = 16
epochs = 32
steps_train = 18
steps_val = 3
img_height = 256
img_width = 256
data_dir='/kaggle/working/images/'
model_merged.compile(loss='categorical_crossentropy',
optimizer="adam",
metrics=['acc'])
train_ds = tf.keras.utils.image_dataset_from_directory(
data_dir,
validation_split=0.25,
subset="training",
seed=123,
image_size=(img_height, img_width),
label_mode="categorical",
batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
data_dir,
validation_split=0.25,
subset="validation",
seed=123,
image_size=(img_height, img_width),
label_mode="categorical",
batch_size=batch_size)

# scale pixel value between 0 and 1
normalization_layer = tf.keras.layers.Rescaling(1./255)
reshape_layer = Reshape((-1,256,256))
resize_layer = Resizing(1,256)
permute_layer = Permute((2,3,1))

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
train_ds = train_ds.map(lambda x, y: (reshape_layer(x), y))
val_ds = val_ds.map(lambda x, y: (reshape_layer(x), y))
train_ds = train_ds.map(lambda x, y: (resize_layer(x), y))
val_ds = val_ds.map(lambda x, y: (resize_layer(x), y))
train_ds = train_ds.map(lambda x, y: (permute_layer(x), y))
val_ds = val_ds.map(lambda x, y: (permute_layer(x), y))


AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

history = model_merged.fit(
train_ds,
steps_per_epoch=steps_train,
epochs=epochs,
validation_data = val_ds,
validation_steps = steps_val,
verbose=1)

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.show()

In [102]:
print("Training Accuracy:"), print(history.history['acc'][-1])
print("Testing Accuracy:"), print (history.history['val_acc'][-1])

Training Accuracy:
0.9947916865348816
Testing Accuracy:
0.71875


(None, None)

In [24]:
# Compute inference time for segmentation
import cv2
import tensorflow as tf
from glob import glob
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from random import randint
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image
import time


PATH = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT"
def process_image(image, HEIGHT=256, WIDTH=256):
    image = image.decode("utf-8")
    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (HEIGHT, WIDTH))
    image = image / 255.0

    image = image.astype(np.float32)
    image = np.expand_dims(image, axis=-1)
    return image
def process_mask(mask):
    mask = mask.astype(np.float32)
    mask = mask / 255
    mask = np.round(mask) 
    mask = np.expand_dims(mask, axis=-1) 
    return mask

def preprocess(x, y):
    def f(x, y):
        image = process_image(x)
        mask = process_mask(y)
        return image, mask

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    image.set_shape([256, 256, 1])
    mask.set_shape([256, 256, 1])
    return image, mask
# combine real masks where there are more masks for the same image
def group_masks(masks):
    new_masks = []
    index = 0
    for i in range(len(masks)):
        m = re.search(r"mask_[0-9]", masks[i]) # we are looking for masks that correspond to the same inputs
        if m:
            new_masks[i - (index + 1)] += cv2.resize(
                cv2.imread(masks[i], cv2.IMREAD_GRAYSCALE), (256, 256)
            )
            index += 1
        else:
            new_masks.append(
                cv2.resize(
                    cv2.imread(masks[i], cv2.IMREAD_GRAYSCALE), (256, 256)
                )  
            )

    return new_masks
benign_images = sorted(
        glob(os.path.join(PATH,"benign/*).png")),
        key=lambda x: [int(c) if c.isdigit() else c for c in re.split(r"(\d+)", x)],
    )
benign_masks = group_masks(sorted(
        glob(os.path.join(PATH, "benign/*mask*")),
        key=lambda x: [int(c) if c.isdigit() else c for c in re.split(r"(\d+)", x)],
    ))
malignant_images = sorted(
        glob(os.path.join(PATH,"malignant/*).png")),
        key=lambda x: [int(c) if c.isdigit() else c for c in re.split(r"(\d+)", x)],
    )
malignant_masks = group_masks(sorted(
        glob(os.path.join(PATH, "malignant/*mask*")),
        key=lambda x: [int(c) if c.isdigit() else c for c in re.split(r"(\d+)", x)],
    ))
normal_images = sorted(
        glob(os.path.join(PATH,"normal/*).png")),
        key=lambda x: [int(c) if c.isdigit() else c for c in re.split(r"(\d+)", x)],
    )
normal_masks = group_masks(sorted(
        glob(os.path.join(PATH, "normal/*mask*")),
        key=lambda x: [int(c) if c.isdigit() else c for c in re.split(r"(\d+)", x)],
    ))
# Version 1: we just train on benign and malignant masks
all_images = benign_images + malignant_images
all_masks = benign_masks + malignant_masks
X_train, X_test = train_test_split(
        all_images,
        test_size=0.2,
        random_state=1,
        shuffle=True
    )
y_train, y_test = train_test_split(
    all_masks,
    test_size=0.2,
    random_state=1,
    shuffle=True
)
batch_size = 16
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.map(preprocess)
train_ds = train_ds.batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.map(preprocess)
test_ds = test_ds.batch(batch_size)

all_ds = train_ds.concatenate(test_ds)
original_images = list(all_ds.map(lambda x, y: y*255))
images_to_mask = list(all_ds.map(lambda x, y: x*255))
start = time.time()
for i in range(10):
    pred = A.predict(images_to_mask[i])
end = time.time()
print(end - start)

1.7141962051391602


In [39]:
from keras.preprocessing.image import ImageDataGenerator
# inference time for classification
predict_datagen =ImageDataGenerator(rescale=1./255)
predict_generator = predict_datagen.flow_from_directory(
    '/kaggle/input/sample-dataset-masti-neaugmentate',  # Source directory
    target_size=(400, 400),  # Resizes images
    batch_size=15,
    class_mode='categorical')

start = time.time()
B.predict_generator(
  predict_generator,
  steps=3,
  max_queue_size = 15,
  workers = 1,
  verbose = 0
)
end = time.time()
print(end - start)

Found 45 images belonging to 3 classes.
0.3506283760070801
